In [9]:
import wave, struct, os
from openai import OpenAI
 # silence deprecation warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
client = OpenAI(api_key="")

In [71]:
responce = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a nice assistant"},
        {"role": "user", "content": "What is the capital of France?"},
    ]
)

In [75]:
responce


ChatCompletion(id='chatcmpl-A7bsttZD0cgH8V0wLykOxrYh878mh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726377287, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_1f2231a495', usage=CompletionUsage(completion_tokens=7, prompt_tokens=23, total_tokens=30, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [77]:
responce.choices[0].message.content

'The capital of France is Paris.'

# Test Scenarios

1. General Knowledge: This category tests the AI model's ability to accurately recall and provide factual information on a wide range of topics.
2. Text Completion: This category assesses the AI model's proficiency in completing sentences or text prompts in a coherent and contextually appropriate manner.
3. Data Analysis: This category evaluates the AI model's capability to perform basic data analysis tasks, such as calculations, statistical operations, and data interpretation.
4. Information Retrieval and Report Generation: This category examines the AI model's ability to extract relevant information and generate concise reports or summaries based on the provided data or prompts.

# OpenAI

In [7]:

import pandas as pd

# Load the test scenarios from the CSV file
test_scenarios_df = pd.read_csv('test_scenarios.csv')

# Display the DataFrame to verify it loaded correctly
print(test_scenarios_df)


                                       Category  \
0                             General Knowledge   
1                             General Knowledge   
2                             General Knowledge   
3                               Text Completion   
4                               Text Completion   
5                               Text Completion   
6                                 Data Analysis   
7                                 Data Analysis   
8                                 Data Analysis   
9   Information Retrieval and Report Generation   
10  Information Retrieval and Report Generation   
11  Information Retrieval and Report Generation   

                                               Prompt  \
0                      What is the capital of France?   
1                  Who wrote 'To Kill a Mockingbird'?   
2              What is the chemical symbol for water?   
3                              The quick brown fox...   
4                        In a galaxy far, far away.

In [28]:
def run_test_scenarios_from_csv(csv_file):
    # Load the CSV file into a DataFrame
    scenarios_df = pd.read_csv(csv_file)
    
    # Iterate over each scenario
    results = []
    for index, row in scenarios_df.iterrows():
        # Extract the prompt and other details
        category = row['Category']
        prompt = row['Prompt']
        expected_output = row['Expected Output']
        
        # Run the prompt with the AI model
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a assistant that only answer questions and skip the process"},
                {"role": "user", "content": prompt}
            ]
        )
        output = response.choices[0].message.content.strip()
        
        # Store the results
        results.append({
            "category": category,
            "prompt": prompt,
            "response": output,
            "expected_output": expected_output,
           # "is_correct": expected_output is None or output.lower() == expected_output.lower()
        })
    
    # Convert results to a DataFrame for easier analysis
    results_df = pd.DataFrame(results)
    return results_df

# Run the test scenarios and display results
results_df = run_test_scenarios_from_csv('test_scenarios.csv')
results_df


,category,prompt,response,expected_output
0,General Knowledge,What is the capital of France?,The capital of France is Paris.,Paris
1,General Knowledge,Who wrote 'To Kill a Mockingbird'?,Harper Lee wrote 'To Kill a Mockingbird'.,Harper Lee
2,General Knowledge,What is the chemical symbol for water?,The chemical symbol for water is H2O.,H2O
3,Text Completion,The quick brown fox...,jumps over the lazy dog.,The quick brown fox jumps over the lazy dog.
4,Text Completion,"In a galaxy far, far away...",Star Wars.,"In a galaxy far, far away, a great adventure w..."
5,Text Completion,Once upon a time in a small village...,"Sorry, I can only answer questions. Do you hav...","Once upon a time in a small village, there liv..."
6,Data Analysis,"Calculate the average of the numbers: 4, 8, 15...","The average of the numbers 4, 8, 15, 16, 23, 4...",18
7,Data Analysis,What is the sum of the first 100 positive inte...,The sum of the first 100 positive integers is ...,5050
8,Data Analysis,"Find the maximum value in the dataset: [12, 47...",The maximum value in the dataset is 85.,85
9,Information Retrieval and Report Generation,Generate a summary report of sales data: 'Prod...,Summary Report:\n- Product A: 500 units sold\n...,"Summary Report: Product A - 500 units, Product..."


In [30]:
results_df.to_csv('OpenAi_results.csv', index=False)


# Anthropic

In [47]:
import anthropic
import pandas as pd

def run_test_scenarios_from_csv_anthropic(csv_file):
    # Load the CSV file into a DataFrame
    scenarios_df = pd.read_csv(csv_file)
    
    # Initialize the Anthropic client (replace 'your_api_key' with the actual API key)
    client = anthropic.Anthropic(api_key='')
    
    # Iterate over each scenario
    results = []
    for index, row in scenarios_df.iterrows():
        # Extract the prompt and other details
        category = row['Category']
        prompt = row['Prompt']
        expected_output = row['Expected Output']

        # Modify the prompt to instruct the model to skip the process
        modified_prompt = f"You are an assistant that only provides final answers and skips the explanation. Here is the query:\n\nHuman: {prompt}\n\nAssistant:"
        
        # Run the prompt with the Anthropic AI model
        response = client.completions.create(
            model="claude-2.1",  
            prompt=modified_prompt,
            max_tokens_to_sample=300  
        )
        output = response.completion.strip()
        
        # Store the results
        results.append({
            "category": category,
            "prompt": prompt,
            "response": output,
            "expected_output": expected_output,
            # "is_correct": expected_output is None or output.lower() == expected_output.lower()
        })
    
    # Convert results to a DataFrame for easier analysis
    results_df = pd.DataFrame(results)
    return results_df

# Run the test scenarios and display results
results_df = run_test_scenarios_from_csv_anthropic('test_scenarios.csv')
results_df


,category,prompt,response,expected_output
0,General Knowledge,What is the capital of France?,The capital of France is Paris.,Paris
1,General Knowledge,Who wrote 'To Kill a Mockingbird'?,Harper Lee wrote the novel 'To Kill a Mockingb...,Harper Lee
2,General Knowledge,What is the chemical symbol for water?,The chemical symbol for water is H2O.,H2O
3,Text Completion,The quick brown fox...,Unfortunately I do not have enough context to ...,The quick brown fox jumps over the lazy dog.
4,Text Completion,"In a galaxy far, far away...",Unfortunately I do not have access to any docu...,"In a galaxy far, far away, a great adventure w..."
5,Text Completion,Once upon a time in a small village...,"*ponders thoughtfully* Ah, a fine beginning to...","Once upon a time in a small village, there liv..."
6,Data Analysis,"Calculate the average of the numbers: 4, 8, 15...","The average of the numbers 4, 8, 15, 16, 23, 4...",18
7,Data Analysis,What is the sum of the first 100 positive inte...,The sum of the first 100 positive integers is ...,5050
8,Data Analysis,"Find the maximum value in the dataset: [12, 47...",85,85
9,Information Retrieval and Report Generation,Generate a summary report of sales data: 'Prod...,Here is the summary report of sales data you r...,"Summary Report: Product A - 500 units, Product..."


In [49]:
results_df.to_csv('Anthropic_results.csv', index=False)

# llama-3-70b

In [27]:
import replicate
import pandas as pd

# Your API token (Not recommended for production usage)
REPLICATE_API_TOKEN = ""

# Set the API token for replicate
replicate.Client(api_token=REPLICATE_API_TOKEN)

def run_test_scenarios_from_csv_replicate(csv_file):
    # Load the CSV file into a DataFrame
    scenarios_df = pd.read_csv(csv_file)

    # Iterate over each scenario
    results = []
    for index, row in scenarios_df.iterrows():
        # Extract the prompt and other details
        category = row['Category']
        prompt = row['Prompt']
        expected_output = row['Expected Output']

        # Modify the prompt to instruct the model to skip the process
        modified_prompt = f"You are an assistant that only provides final answers and skips the explanation. Here is the query:\n\n{prompt}"

        # Define the input parameters for the LLaMA model via Replicate
        input_params = {
            "top_k": 50,
            "top_p": 0.9,
            "prompt": modified_prompt,
            "max_tokens": 300,  # Adjust token length if needed
            "temperature": 0.6,
            "presence_penalty": 1.15,
            "frequency_penalty": 0.2,
        }

        # Run the LLaMA model using Replicate API and stream the output
        response = ""
        try:
            for event in replicate.stream("meta/meta-llama-3-70b", input=input_params):
                response += str(event)  # Collect the streaming output
        except Exception as e:
            response = f"Error: {str(e)}"

        # Store the results
        results.append({
            "category": category,
            "prompt": prompt,
            "response": response,
            "expected_output": expected_output,
        })
    
    # Convert results to a DataFrame for easier analysis
    results_df = pd.DataFrame(results)
    return results_df

# Run the test scenarios and display results
results_df = run_test_scenarios_from_csv_replicate('test_scenarios.csv')
print(results_df)


                                       category  \
0                             General Knowledge   
1                             General Knowledge   
2                             General Knowledge   
3                               Text Completion   
4                               Text Completion   
5                               Text Completion   
6                                 Data Analysis   
7                                 Data Analysis   
8                                 Data Analysis   
9   Information Retrieval and Report Generation   
10  Information Retrieval and Report Generation   
11  Information Retrieval and Report Generation   

                                               prompt  \
0                      What is the capital of France?   
1                  Who wrote 'To Kill a Mockingbird'?   
2              What is the chemical symbol for water?   
3                              The quick brown fox...   
4                        In a galaxy far, far away.

In [23]:
results_df.to_csv('Llama70b_results.csv', index=False)